# 划分训练集和验证集

In [2]:
import os
import shutil
import random

按照MMPreTrain CustomDataset格式组织训练集和验证集

In [14]:
raw_data_path = 'fruit30_train'
data_path = 'fruit30'
fruits = os.listdir(raw_data_path)
os.makedirs(data_path, exist_ok = True)
train_rate = 0.8

In [19]:
for fruit in fruits:
    fruit_path_train = os.path.join(data_path, 'train', fruit)
    fruit_path_val = os.path.join(data_path, 'val', fruit)
    os.makedirs(fruit_path_train, exist_ok=True)
    os.makedirs(fruit_path_val, exist_ok=True)
    fruit_pics = os.listdir(os.path.join(raw_data_path, fruit))
    random.shuffle(fruit_pics)
    train_index = int(len(fruit_pics) * train_rate)
    for fruit_pic in fruit_pics[: train_index]:
        shutil.copy(os.path.join(raw_data_path, fruit, fruit_pic), os.path.join(fruit_path_train, fruit_pic))
    for fruit_pic in fruit_pics[train_index :]:
        shutil.copy(os.path.join(raw_data_path, fruit, fruit_pic), os.path.join(fruit_path_val, fruit_pic))

# 使用MMPreTrain算法库，编写配置文件，正确加载预训练模型，并在水果数据集上微调

In [18]:
!pip install opencv-python-headless
!mim train mmpretrain configs/resnet50_8xb32_in1k.py --work-dir=./log/train

Training command is /home/leo/miniconda3/envs/openmmlab/bin/python /home/leo/Code/mmpretrain/mmpretrain/.mim/tools/train.py configs/resnet50_8xb32_in1k.py --launcher none --work-dir=./log/train. 
06/11 22:39:49 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.11 (main, May 16 2023, 00:28:57) [GCC 11.2.0]
    CUDA available: True
    numpy_random_seed: 1632906121
    GPU 0: NVIDIA GeForce GTX 1660 Ti
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.8, V11.8.89
    GCC: gcc (Ubuntu 11.3.0-1ubuntu1~22.04.1) 11.3.0
    PyTorch: 2.0.1
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2023.1-Product Build 20230303 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enab

# 使用MMPreTrain对水果图像使用训练好的模型进行分类

In [20]:
from mmpretrain import ImageClassificationInferencer
inferencer = ImageClassificationInferencer('./configs/resnet50_8xb32_in1k.py', pretrained='./log/train/epoch_17.pth', device='cuda:0')
inferencer(['./test_fruit/watermelon.jpg'], show_dir="./test_fruit")

Loads checkpoint by local backend from path: ./log/train/epoch_17.pth
06/11 23:13:57 - mmengine - WARNING - `Visualizer` backend is not initialized because save_dir is None.


/home/leo/miniconda3/envs/openmmlab/lib/python3.10/site-packages/mmengine/visualization/utils.py:240: UserWarning: Glyph 35199 (\N{CJK UNIFIED IDEOGRAPH-897F}) missing from current font.
  s, (width, height) = canvas.print_to_buffer()
/home/leo/miniconda3/envs/openmmlab/lib/python3.10/site-packages/mmengine/visualization/utils.py:240: UserWarning: Glyph 29916 (\N{CJK UNIFIED IDEOGRAPH-74DC}) missing from current font.
  s, (width, height) = canvas.print_to_buffer()


[{'pred_scores': array([1.6223394e-08, 4.5852357e-09, 9.4088938e-11, 3.1583698e-09,
         8.6965451e-07, 2.1511442e-09, 3.0585045e-14, 4.8741455e-10,
         1.1022438e-06, 2.7802785e-10, 6.4201984e-08, 9.8760444e-10,
         8.3397698e-09, 9.1619134e-12, 1.3811403e-09, 1.2690999e-11,
         7.5741946e-09, 7.7462681e-10, 1.5615097e-07, 1.2396989e-11,
         1.1202120e-07, 7.2704942e-10, 3.3101986e-11, 2.3659495e-11,
         2.7744541e-08, 9.9999034e-01, 7.2518606e-06, 1.7536744e-09,
         2.9624658e-10, 3.1454983e-09], dtype=float32),
  'pred_label': 25,
  'pred_score': 0.9999903440475464,
  'pred_class': '西瓜'}]